<a href="https://colab.research.google.com/github/kifjj/altinha-play/blob/main/alta_train_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


```markdown
To remove a non-empty folder (directory) and all its contents in Python, you can use the `shutil.rmtree()` function. Be very careful when using this, as it permanently deletes files and folders, and there's no undo. Always double-check the path you are providing.
```

In [ ]:
#python
import shutil
import os

# Define the path to the folder you want to remove
# Replace 'path/to/your/folder' with the actual path
folder_to_remove = '/content/runs'

# IMPORTANT: Make sure this path is correct before running!
# For example, let's create a dummy folder to demonstrate
# os.makedirs(os.path.join(folder_to_remove, 'subfolder'), exist_ok=True)
# with open(os.path.join(folder_to_remove, 'file.txt'), 'w') as f:
#     f.write('test content')

if os.path.exists(folder_to_remove):
    try:
        shutil.rmtree(folder_to_remove)
        print(f"Folder '{folder_to_remove}' and its contents removed successfully.")
    except OSError as e:
        print(f"Error: {folder_to_remove} : {e.strerror}")
else:
    print(f"Folder '{folder_to_remove}' does not exist.")

# Uncomment the following lines and replace with your actual folder path to use it:
# # For example, to remove a folder named 'my_empty_folder' from '/content/'
# # my_folder_path = '/content/my_empty_folder'
# # if os.path.exists(my_folder_path):
# #     try:
# #         shutil.rmtree(my_folder_path)
# #         print(f"Folder '{my_folder_path}' and its contents removed successfully.")
# #     except OSError as e:
# #         print(f"Error: {my_folder_path} : {e.strerror}")
# # else:
# #     print(f"Folder '{my_folder_path}' does not exist.")

Folder '/content/runs' and its contents removed successfully.


In [ ]:
import zipfile
import os

# Path to your zip in Drive
zip_path = 'frames_pic'

# Where to extract (Colab's local disk)
extract_path = 'frames'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset ready at:", extract_path)

FileNotFoundError: [Errno 2] No such file or directory: 'frames_pic'

In [ ]:
import zipfile
import os

# Define the path to the folder to be zipped
source_folder = '/content/frames'
# Define the name of the output zip file
output_zip_file = '/content/frames.zip'

# Check if the source folder exists
if os.path.exists(source_folder) and os.path.isdir(source_folder):
    with zipfile.ZipFile(output_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to zip, preserving directory structure relative to source_folder
                zipf.write(file_path, os.path.relpath(file_path, source_folder))
    print(f"Successfully created '{output_zip_file}' containing all files from '{source_folder}'.")
else:
    print(f"Error: The folder '{source_folder}' does not exist or is not a directory.")

Successfully created '/content/frames.zip' containing all files from '/content/frames'.


In [ ]:
import os

current_directory = os.getcwd()
print(f"The current folder is: {current_directory}")

The current folder is: /content


In [ ]:
import zipfile
import os

# Define the path to the folder to be zipped
source_folder = '/content/frames'
# Define the name of the output zip file
output_zip_file = '/content/frames.zip'

# Check if the source folder exists
if os.path.exists(source_folder) and os.path.isdir(source_folder):
    with zipfile.ZipFile(output_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to zip, preserving directory structure relative to source_folder
                zipf.write(file_path, os.path.relpath(file_path, source_folder))
    print(f"Successfully created '{output_zip_file}' containing all files from '{source_folder}'.")
else:
    print(f"Error: The folder '{source_folder}' does not exist or is not a directory.")


Successfully created '/content/frames.zip' containing all files from '/content/frames'.


In [ ]:
!pip install ultralytics

from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
# model = YOLO('yolov8l.pt')

print("Ultralytics library installed and YOLOv8n model loaded.")

In [ ]:
from ultralytics import YOLO

# 1. Load the model (start with Medium for a balance of speed/accuracy)
model = YOLO('yolo11m.pt')

# 2. Train
results = model.train(
    # Point to the data.yaml file inside your unzipped folder
    data='/content/datasets/data.yaml',

    epochs=100,             # 100 is a good sweet spot
    imgsz=1280,             # High resolution training
    batch=4,                # Lower batch size to save GPU memory (increase to 8 if stable)

    cache=True,  # <--- ADDS SPEED (if you have enough RAM)
    workers=8,    # <--- Use 8 workers to load data faster

    # SPECIAL SAUCE FOR ALTINHA
    mosaic=0.0,             # Disable mosaic
    mixup=0.0,              # Disable mixup
    name='altinha_v1'       # Project name
)

In [ ]:
import shutil

# The result is usually saved in runs/detect/altinha_v1/weights/best.pt
source = '/content/runs/detect/altinha_v1/weights/best.pt'
destination = '/content/altinha_best.pt'

shutil.copy(source, destination)
print(f"Saved model to Drive: {destination}")

Saved model to Drive: /content/altinha_best.pt


In [ ]:
!pip install --upgrade sahi ultralytics supervision

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

# --- CONFIGURATION ---
VIDEO_PATH = '/content/alta-champion-open-8s.mp4'
# Use the same fine-tuned weights to keep the comparison fair
MODEL_PATH = '/content/altinha_best.pt'
OUTPUT_PATH = '/content/altinha_standard_result.mp4'
CONFIDENCE = 0.25
# ---------------------

# 1. Initialize Model
# We load the model directly using Ultralytics (no SAHI wrapper)
model = YOLO(MODEL_PATH)

# 2. Initialize Tracker & Annotators
# NEW (Correct code for supervision >= 0.25.0)
tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    minimum_matching_threshold=0.8,
    frame_rate=30
)
box_annotator = sv.BoxAnnotator(thickness=3, color=sv.Color.from_hex("#00FF00"))
label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=2, text_position=sv.Position.TOP_CENTER)
trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=30)

# 3. Video Processing
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"Processing {VIDEO_PATH} with Standard Inference...")

with sv.VideoSink(target_path=OUTPUT_PATH, video_info=video_info) as sink:
    for frame in frames_generator:

        # A. STANDARD INFERENCE (The Fast Way)
        # Instead of slicing, we pass the whole frame at once.
        # verbose=False keeps your console clean.
        results = model(frame, verbose=False, conf=CONFIDENCE)[0]

        # B. CONVERT TO SUPERVISION
        # Supervision has a built-in helper for standard YOLO results
        detections = sv.Detections.from_ultralytics(results)

        # Filter specifically for class 0 (if you have only 1 class)
        # or the specific class ID for 'ball' if you kept original COCO classes
        # detections = detections[detections.class_id == 0]

        # C. UPDATE TRACKER
        detections = tracker.update_with_detections(detections)

        # D. DRAW
        annotated_frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

        labels = [
            f"Ball #{tracker_id} {conf:.2f}"
            for tracker_id, conf
            in zip(detections.tracker_id, detections.confidence)
        ]
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )

        sink.write_frame(annotated_frame)

print(f"Done! Video saved to {OUTPUT_PATH}")

Processing /content/alta-champion-open-8s.mp4 with Standard Inference...
Done! Video saved to /content/altinha_standard_result.mp4


In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

# --- CONFIGURATION ---
VIDEO_PATH = '/content/altinha-beach-green-mq.mp4'
# Use the same fine-tuned weights to keep the comparison fair
MODEL_PATH = '/content/altinha_best.pt'
OUTPUT_PATH = '/content/altinha-beach-green-mq-SAHI.mp4'
CONFIDENCE = 0.001
iou=0.9
# ---------------------

# 1. Initialize Model
# We load the model directly using Ultralytics (no SAHI wrapper)
model = YOLO(
      MODEL_PATH)


# 2. Initialize Tracker & Annotators

tracker = sv.ByteTrack(
    track_activation_threshold=0.1,
    minimum_matching_threshold=0.3,
    frame_rate=30
)

box_annotator = sv.BoxAnnotator(thickness=3, color=sv.Color.from_hex("#00FF00"))
label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=2, text_position=sv.Position.TOP_CENTER)
trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=30)

# 3. Video Processing
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"Processing {VIDEO_PATH} with Standard Inference...")

with sv.VideoSink(target_path=OUTPUT_PATH, video_info=video_info) as sink:
    n_frame = 0
    for frame in frames_generator:
        n_frame += 1
        cv2.imwrite(f"/content/frames/frame_{n_frame}.png", frame)
        # A. STANDARD INFERENCE (The Fast Way)
        # Instead of slicing, we pass the whole frame at once.
        # verbose=False keeps your console clean.
        results = model(
            frame,
            verbose=False,
            conf=CONFIDENCE,
            iou=iou
            )[0]

        # B. CONVERT TO SUPERVISION
        # Supervision has a built-in helper for standard YOLO results
        detections = sv.Detections.from_ultralytics(results)

        # Filter specifically for class 0 (if you have only 1 class)
        # or the specific class ID for 'ball' if you kept original COCO classes
        # detections = detections[detections.class_id == 0]

        # C. UPDATE TRACKER
        detections = tracker.update_with_detections(detections)

        # D. DRAW
        annotated_frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
        annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)

        labels = [
            f"Ball #{tracker_id} {conf:.2f}"
            for tracker_id, conf
            in zip(detections.tracker_id, detections.confidence)
        ]
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )

        sink.write_frame(annotated_frame)

print(f"Done! Video saved to {OUTPUT_PATH}")

Processing /content/altinha-beach-green-mq.mp4 with Standard Inference...
Done! Video saved to /content/altinha-beach-green-mq-SAHI.mp4


In [ ]:
model = YOLO(MODEL_PATH)

img = cv2.imread("/content/frames/frame_280.png")
results = model(img, conf=0.001, iou=0.9, verbose=False)[0]
# print(results.boxes)

boxes = results.boxes  # Boxes object

for i in range(len(boxes)):
    x1, y1, x2, y2, conf, cls_id = boxes.data[i].tolist()
    print(f"Det {i}: cls={int(cls_id)}, conf={conf:.4f}, "
          f"x1={x1:.1f}, y1={y1:.1f}, x2={x2:.1f}, y2={y2:.1f}")

best = boxes[0]  # highest conf
x1, y1, x2, y2 = map(int, best.xyxy[0].tolist())

print(x1, y1, x2, y2, float(best.conf[0]))

crop = img[y1:y2, x1:x2]
cv2.imwrite("debug_crop.png", crop)

Det 0: cls=0, conf=0.8425, x1=428.1, y1=166.8, x2=468.4, y2=210.6
Det 1: cls=0, conf=0.8153, x1=426.8, y1=168.6, x2=470.4, y2=208.4
Det 2: cls=0, conf=0.6414, x1=428.5, y1=168.1, x2=467.5, y2=206.3
Det 3: cls=0, conf=0.5939, x1=433.1, y1=168.5, x2=468.7, y2=202.3
Det 4: cls=0, conf=0.5526, x1=429.8, y1=167.4, x2=469.2, y2=205.7
Det 5: cls=0, conf=0.5325, x1=432.2, y1=171.5, x2=469.6, y2=203.6
Det 6: cls=0, conf=0.4324, x1=430.9, y1=169.6, x2=466.9, y2=202.7
Det 7: cls=0, conf=0.3614, x1=428.6, y1=165.8, x2=471.7, y2=208.0
Det 8: cls=0, conf=0.2704, x1=442.9, y1=176.5, x2=469.4, y2=196.2
Det 9: cls=0, conf=0.2631, x1=428.2, y1=172.7, x2=472.4, y2=210.9
Det 10: cls=0, conf=0.2397, x1=434.9, y1=171.1, x2=468.4, y2=201.6
Det 11: cls=0, conf=0.1513, x1=433.8, y1=174.8, x2=471.5, y2=207.5
Det 12: cls=0, conf=0.1432, x1=429.5, y1=166.6, x2=465.1, y2=199.5
Det 13: cls=0, conf=0.1308, x1=443.7, y1=173.2, x2=469.4, y2=195.5
Det 14: cls=0, conf=0.1299, x1=444.1, y1=173.8, x2=469.9, y2=199.6
Det 1

True

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

# VIDEO_PATH = '/content/altinha-beach-green-mq.mp4'
# VIDEO_PATH = '/content/alta-champion-open-8s.mp4'
VIDEO_PATH = '/content/altinha-beach-green-mq.mp4'


MODEL_PATH = '/content/altinha_best.pt'
OUTPUT_PATH = '/content/altinha-beach-green-mq-BEST_ONLY.mp4'

CONFIDENCE = 0.05   # raise this a bit now that we don’t need raw debug
IOU_NMS   = 0.5     # normal-ish NMS now


# Hit detection hyperparams
MIN_VERTICAL_AMPLITUDE = 3   # pixels, how "deep" the local max should be
MIN_FRAMES_BETWEEN_HITS = 8   # avoid double-counting almost identical frames
GAP_RESET_FRAMES = 30         # e.g. 1 second at 30 fps
SAVE_FRAMES = False           # set True if you still want frame_XXX.png

model = YOLO(MODEL_PATH)

box_annotator = sv.BoxAnnotator(
    thickness=2,
    color=sv.Color.from_hex("#00FF00")  # just one box = the chosen ball
)
label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_thickness=2,
    text_position=sv.Position.TOP_CENTER,
)

video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
fps = video_info.fps
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"Processing {VIDEO_PATH} using BEST detection per frame...\n")

# --- State for hit detection ---
# We'll keep the last 3 (frame_idx, y_center) where we had a detection
last_positions = []         # list of (frame_idx, y_center)
hit_frames = []             # list of frame_idx where we detected a hit
last_detection_frame = None # last frame index where we had ANY detection

with sv.VideoSink(target_path=OUTPUT_PATH, video_info=video_info) as sink:
    n_frame = 0
    for frame in frames_generator:
        n_frame += 1

        results = model(
            frame,
            verbose=False,
            conf=CONFIDENCE,
            iou=IOU_NMS,
        )[0]

        detections = sv.Detections.from_ultralytics(results)

        # If there are detections, keep only the best one (highest confidence)
        if len(detections) > 0:
            best_idx = int(np.argmax(detections.confidence))

            best_conf = float(detections.confidence[best_idx])

            if best_conf >= 0.08:
                # keep this detection
                detections = detections[best_idx:best_idx+1]  # slice keeps Detections object
            else:
                # treat as "no detection" for this frame
                detections = detections[0:0]  # empty Detections object


        else:
            # keep detections empty
            pass

        # Debug (optional)
        if len(detections) > 0:
            conf = float(detections.confidence[0])
            x1, y1, x2, y2 = detections.xyxy[0].tolist()

            y_center = 0.5 * (y1 + y2)

            # 1) Check for a long gap since last detection
            if last_detection_frame is not None:
                gap = n_frame - last_detection_frame
                if gap > GAP_RESET_FRAMES:
                    # Ball was off-screen (or undetected) for a long time:
                    # reset the local history so we don't accidentally
                    # connect trajectories across that gap.
                    last_positions.clear()

            last_detection_frame = n_frame


            last_positions.append((n_frame, y_center))
            if len(last_positions) > 3:
                last_positions.pop(0)  # keep only last 3


            # once we have 3 points, we can test the middle one for local max
            # 3) If we have 3 points, test middle one for local max of y
            if len(last_positions) == 3:
                (f0, y0), (f1, y1c), (f2, y2c) = last_positions

                going_down_then_up = (y0 < y1c) and (y2c < y1c)
                vertical_span = y1c - min(y0, y2c)

                if going_down_then_up and vertical_span >= MIN_VERTICAL_AMPLITUDE:
                    # enforce minimum gap between hits (to avoid double-counting
                    # small jitter near the same contact)
                    if not hit_frames or (f1 - hit_frames[-1]) >= MIN_FRAMES_BETWEEN_HITS:
                        hit_frames.append(f1)
                        t_sec = f1 / fps
                        print(
                            f"HIT detected at frame {f1} "
                            f"(t ≈ {t_sec:.2f}s), y={y1c:.1f}, span={vertical_span:.1f}"
                        )


            print(
                f"FRAME {n_frame:4d}: best conf={conf:.3f}, "
                f"bbox=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})"
            )
        else:
            print(f"FRAME {n_frame:4d}: no detection")

        # Draw
        # --- DRAW ---
        annotated_frame = frame.copy()

        # Draw ball box + confidence
        if len(detections) > 0:
            conf = float(detections.confidence[0])
            labels = [f"Ball {conf:.2f}"]
            annotated_frame = box_annotator.annotate(
                scene=annotated_frame,
                detections=detections,
            )
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame,
                detections=detections,
                labels=labels,
            )

        # === TOP-LEFT HUD BOX WITH HIT COUNT ===
        hit_text = f"Hits: {len(hit_frames)}"

        # choose font & scale
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        thickness = 2

        # measure text size
        (text_width, text_height), baseline = cv2.getTextSize(
            hit_text, font, font_scale, thickness
        )

        # box position + padding
        pad_x, pad_y = 10, 10
        x1, y1 = 10, 10  # top-left corner of the box
        x2 = x1 + text_width + 2 * pad_x
        y2 = y1 + text_height + 2 * pad_y

        # draw filled rectangle (semi-opaque-ish look)
        cv2.rectangle(
            annotated_frame,
            (x1, y1),
            (x2, y2),
            (0, 0, 0),     # black box
            thickness=-1   # filled
        )

        # draw the text inside the box
        text_x = x1 + pad_x
        text_y = y1 + pad_y + text_height  # baseline is at bottom of text
        cv2.putText(
            annotated_frame,
            hit_text,
            (text_x, text_y),
            font,
            font_scale,
            (0, 255, 0),  # green text
            thickness,
            cv2.LINE_AA,
        )
        # =======================================

        sink.write_frame(annotated_frame)

print(f"\nDone! Video saved to {OUTPUT_PATH}")

print(f"Estimated number of hits/passes: {len(hit_frames)}")

# Also print times for manual inspection
print("\nHit frames and timestamps:")
for f in hit_frames:
    print(f"  frame {f}, t ≈ {f / fps:.2f}s")


Processing /content/altinha-beach-green-mq.mp4 using BEST detection per frame...

FRAME    1: best conf=0.959, bbox=(681.1,109.9,719.0,148.5)
FRAME    2: best conf=0.957, bbox=(651.6,108.9,689.3,147.0)
FRAME    3: best conf=0.920, bbox=(623.5,109.0,658.9,147.5)
FRAME    4: best conf=0.962, bbox=(590.7,109.7,628.6,149.6)
FRAME    5: best conf=0.954, bbox=(560.0,112.8,597.6,151.6)
FRAME    6: best conf=0.777, bbox=(531.5,118.3,567.5,155.7)
FRAME    7: best conf=0.812, bbox=(497.8,124.4,537.5,162.6)
FRAME    8: best conf=0.888, bbox=(470.6,131.4,508.5,170.2)
FRAME    9: best conf=0.883, bbox=(439.5,141.1,478.8,176.1)
FRAME   10: best conf=0.936, bbox=(411.1,149.9,448.4,185.5)
FRAME   11: best conf=0.901, bbox=(381.9,160.9,418.2,197.1)
FRAME   12: best conf=0.911, bbox=(348.5,174.6,388.7,213.1)
FRAME   13: best conf=0.945, bbox=(319.9,189.9,359.2,225.6)
FRAME   14: best conf=0.945, bbox=(292.3,206.6,330.4,244.7)
FRAME   15: best conf=0.919, bbox=(262.9,226.2,302.0,265.8)
FRAME   16: best c